In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

from datasets import Dataset
from functools import partial
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction

from src.utils import transform_start_field
from src import ts_transformer as tsf
from src.inference.wrapper import TFWrapper
from src.ts_transformer import create_train_dataloader

from src.networks.classifier import RepresentationClassifier, train_classifier, train_joint_sequential

# Get Dataset

In [ ]:
#assumes a dataset <data>

train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
print(train_df.shape)

# Load Model

In [ ]:
# Loading a pretrained model, for training check train_test.ipynb

freq = '1H'
transformer = TimeSeriesTransformerForPrediction.from_pretrained(
    "<PATH_TO_WEIGHTS>")
model = TFWrapper(transformer, freq)

# Classifier

1. forward pass of all data in trained transformer
2. extract latent space
3. add corresponding label
4. Jointly train projection network and classifier

In [ ]:
train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
print(train_df.shape)

In [ ]:
train_data = Dataset.from_pandas(train_df, preserve_index=False)
train_data.set_transform(partial(transform_start_field, freq=freq))

valid_data = Dataset.from_pandas(valid_df, preserve_index=False)
valid_data.set_transform(partial(transform_start_field, freq=freq))

In [ ]:
train_dataloader = create_train_dataloader(
                            config=model.model_config,
                            freq=model.freq,
                            data=train_data,
                            batch_size=32,
                            num_batches_per_epoch=16)

valid_dataloader = create_train_dataloader(
                            config=model.model_config,
                            freq=model.freq,
                            data=valid_data,
                            batch_size=32,
                            num_batches_per_epoch=16)

In [ ]:
num_epochs = 50

# we classify the first static feature
num_classes = len(list(set([i[0] for i in train_df['feat_static_cat']])))

classifier = RepresentationClassifier(
    encoder_hidden_size=model.model_config.d_model,
    attn_hidden_dims= None, #[64, 32],
    classifier_hidden_dims= None, #[64, 32],
    num_classes=num_classes,
    attn_activation=nn.Tanh,
    classifier_activation=nn.ReLU,
    attn_dropout=0,
    classifier_dropout=0,
)

optimizer = optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [ ]:
classifier, train_losses, valid_losses = train_classifier(
    model=model.transformer,
    classifier=classifier,
    train_loader=train_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    device=model.transformer.device,
    num_epochs=num_epochs,
    valid_loader=valid_dataloader)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(train_losses, label='Classifier Training Loss')
plt.plot(valid_losses, label='Classifier Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

# Joint Transformer Classifier Training

In [ ]:
freq = '1H'

transformer_config = {
    'prediction_length': 24,
    'context_length': 48,
    'num_static_categorical_features': 2,
    'cardinality': [3,4],
    'embedding_dimension': [2,2],
    'encoder_layers': 4,
    'decoder_layers': 4,
    'd_model': 32,
    'output_hidden_states': True
}

transformer, train_dataloader = tsf.setup_training(
    train_df=train_df,
    freq=freq,
    batch_size = 32,
    num_batches_per_epoch = 16,
    max_lags = len(train_df['target'][0]) - transformer_config['context_length'],
    transformer_config=transformer_config
)

num_classes = len(list(set([i[0] for i in train_df['feat_static_cat']])))

classifier = RepresentationClassifier(
    encoder_hidden_size=transformer_config['d_model'],
    attn_hidden_dims= None, #[64, 32],
    classifier_hidden_dims= None, #[64, 32],
    num_classes=num_classes,
    attn_activation=nn.Tanh,
    classifier_activation=nn.ReLU,
    attn_dropout=0,
    classifier_dropout=0,
)

optimizer_classifier = optim.Adam(classifier.parameters(), lr=1e-3)
classifier_criterion = nn.CrossEntropyLoss()

optimizer_ts = optim.AdamW(transformer.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

In [ ]:
transformer, classifier, transformer_loss_history, classifier_loss_history = train_joint_sequential(
    transformer,
    classifier,
    train_dataloader, # Single dataloader for both tasks
    optimizer_classifier,
    classifier_criterion,
    optimizer_ts,
    device="cuda",
    num_epochs=400)

In [ ]:
epochs = range(1, len(transformer_loss_history) + 1)

fig, ax1 = plt.subplots()

ax1.plot(epochs, transformer_loss_history, 'b-', label='Transformer Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Transformer Loss')
ax1.tick_params(axis='y')

ax2 = ax1.twinx()
ax2.plot(epochs, classifier_loss_history, 'r-', label='Classifier Loss')
ax2.set_ylabel('Classifier Loss')
ax2.tick_params(axis='y')

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')

plt.title("Training Losses (Transformer and Classifier)")
plt.show()